# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

from ipywidgets.embed import embed_minimal_html

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
Cities_DF = pd.read_csv('../output_data/Cities.csv')
Cities_DF.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
#configure gmaps
gmaps.configure(api_key=g_key)

#get locations and humidity
city_loc = Cities_DF[['Lat','Lng']].astype(float)
humidity = Cities_DF['Humidity'].astype(float)

#format the layout
layout = {
    'width':'1000px',
    'height':'600px',
    'border':'lpx solid black',
    'padding':'lpx'}

#adding heatmap
fig = gmaps.figure(map_type='HYBRID',layout=layout)
heat_layer = gmaps.heatmap_layer(city_loc,weights=humidity,dissipating=False,max_intensity=100,point_radius=1)

fig.add_layer(heat_layer)
fig
#plt.savefig("../output_data/humidity_heatmap.png")



Figure(layout=FigureLayout(border='lpx solid black', height='600px', padding='lpx', width='1000px'))

I included file 'map_heatmap.png' of the hotel maps in the output_data folder to make sure its the required angle.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
myweather = Cities_DF.loc[(Cities_DF['Max Temp'] >= 70) & (Cities_DF['Max Temp'] <= 80) &
                          (Cities_DF['Wind Speed'] <= 10) & (Cities_DF['Cloudiness'] == 0)]
myweather    

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = myweather
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [35]:
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params["location"] = f'{lat},{lng}'
        
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing field/result for {city}... skipping.")
        
    print("------------")

<ipython-input-35-0244873efeee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


Closest opuwo hotel is Opuwo Country Lodge.
------------


/Users/Pulast/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest ponta do sol hotel is Riviera Capitólio Hotel, o mais próximo das cachoeiras (cânions, paraíso perdido, trilha do sol).
------------
Closest cidreira hotel is Hotel Araçá.
------------
Closest morondava hotel is Kimony Resort Hotel.
------------
Closest vaini hotel is DANDELI CROCODILE EDGE HOME STAY.
------------
Closest nador hotel is Hôtel Mercure Rif Nador.
------------
Closest mogok hotel is MOGOK HILL HOTEL.
------------
Closest birjand hotel is مهمان پذیر نوید.
------------


In [36]:
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df = hotel_df.rename(columns={"Lat":"Lat","Lng":"Lng"})
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,"Riviera Capitólio Hotel, o mais próximo das ca..."
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Araçá
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Kimony Resort Hotel
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,DANDELI CROCODILE EDGE HOME STAY
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Mercure Rif Nador
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,مهمان پذیر نوید


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='lpx solid black', height='600px', padding='lpx', width='1000px'))

I included file 'hotels_map_heatmap.png' of the hotel maps in the output_data folder to make sure its the required angle.